In [1]:
import sys, os, re
import pandas as pd
import pyranges as pr
import csv

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None) 

In [2]:
insertions_data = pd.read_csv("filtered_insertions.tsv", sep="\t")

In [3]:
insertions_data.head()

,sample_name,contig,chrA,coordA,orientA,chrB,coordB,orientB,prelim.primary_brkpt_type,prelim.num_primary_reads,prelim.num_supp_reads,prelim.mean_hits,prelim.mean_min_per_id,prelim.mean_max_end_clipping,prelim.mean_min_anchor_len,prelim.total,prelim.adj_total,prelim.frac_reads_removed,entropyA,entropyB,flankA,flankB,splice_type,virus_brkend_grp,is_primary,split,span,total,flankA_fU,flankB_fU,TCGA,virus,humanchr,i_site
0,CESC-C5-A1M6-TP,HPV18~930~+~chr18~39949673~+,HPV18,930,+,chr18,39949673,+,Split,231,857,4.821,98.5,6.337,37.156,1088,1088,0.0,1.889,1.984,agcagtaagcaacaatggctgatccagaagGT,AGtttggagatcttccaccaatatgcccagtg,GT-AG,HPV18:930,True,62.0,527.0,589,1.000,1.000,CESC,HPV18,chr18,HPV18:chr18
1,CESC-C5-A1M6-TP,HPV18~234~+~chr18~39957192~+,HPV18,234,+,chr18,39957192,+,Span,1,8,1.111,97.7,2.333,41.889,9,9,0.0,1.917,1.859,tattgcaagacagtattggaacttacagagGT,CTagagatctgtggaactttgaacttaagaaa,.,HPV18:234,Maybe,1.0,240.0,241,0.963,0.926,CESC,HPV18,chr18,HPV18:chr18
2,CESC-C5-A1M6-TP,HPV18~234~+~chr18~39949673~+,HPV18,234,+,chr18,39949673,+,Split,7,17,7.042,99.9,8.208,34.667,24,24,0.0,1.917,1.984,tattgcaagacagtattggaacttacagagGT,AGtttggagatcttccaccaatatgcccagtg,GT-AG,HPV18:234,True,1.0,14.0,15,0.963,1.000,CESC,HPV18,chr18,HPV18:chr18
3,CESC-C5-A7UC-TP,HPV45~930~+~chr17~59735337~+,HPV45,930,+,chr17,59735337,+,Split,138,1160,1.032,98.6,4.871,42.690,1298,1298,0.0,1.914,1.914,accaataatctacaatggcggatccagaagGT,AGacccctcttcagtgaatgaaaagaagagga,GT-AG,HPV45:930,True,151.0,728.0,879,1.000,1.000,CESC,HPV45,chr17,HPV45:chr17
4,CESC-C5-A7UC-TP,HPV45~231~+~chr17~59735337~+,HPV45,231,+,chr17,59735337,+,Split,9,116,1.048,99.3,2.936,44.392,125,125,0.0,1.914,1.914,tattgcaaagcaacattggaacgcacagagGT,AGacccctcttcagtgaatgaaaagaagagga,GT-AG,HPV45:231,True,6.0,82.0,88,1.000,1.000,CESC,HPV45,chr17,HPV45:chr17


In [4]:
insertions_data.columns

Index(['sample_name', 'contig', 'chrA', 'coordA', 'orientA', 'chrB', 'coordB',
       'orientB', 'prelim.primary_brkpt_type', 'prelim.num_primary_reads',
       'prelim.num_supp_reads', 'prelim.mean_hits', 'prelim.mean_min_per_id',
       'prelim.mean_max_end_clipping', 'prelim.mean_min_anchor_len',
       'prelim.total', 'prelim.adj_total', 'prelim.frac_reads_removed',
       'entropyA', 'entropyB', 'flankA', 'flankB', 'splice_type',
       'virus_brkend_grp', 'is_primary', 'split', 'span', 'total', 'flankA_fU',
       'flankB_fU', 'TCGA', 'virus', 'humanchr', 'i_site'],
      dtype='object')

In [21]:
fusions_data = pd.read_csv("data/TCGA.STAR-Fusion.v1.7.fusion_preds.tsv.gz", sep="\t")
fusions_data.rename(columns={'sample':'sample_name'}, inplace=True)

In [22]:
fusions_data.head()

,data_class,tissue_type,tissue_subtype,tumor_subtype,sample_name,fusion_name,Jcount,Scount,Tcount,SpliceType,LeftGene,LeftBreakpoint,RightGene,RightBreakpoint,LargeAnchorSupport,FFPM,LeftBreakDinuc,LeftBreakEntropy,RightBreakDinuc,RightBreakEntropy,Annots,PROT_FUSION_TYPE,tumor_or_normal,is_primary,cosmic,structure_type
0,TCGA,OV,"Serous cystadenocarcinoma, NOS",TP,OV-24-1428-TP,SEPT12::SEPT9,2.0,0.0,2,ONLY_REF_SPLICE,SEPT12^ENSG00000140623.12,chr16:4779690:-,SEPT9^ENSG00000184640.16,chr17:77497315:+,YES_LDAS,0.0092,GT,1.8892,AG,1.6895,"[HGNC_GENEFAM,INTERCHROMOSOMAL[chr16--chr17]]",INFRAME,tumor,True,False,INTERCHR
1,TCGA,OV,"Serous cystadenocarcinoma, NOS",TP,OV-24-1428-TP,CCDC14::MCCC1,2.0,0.0,2,ONLY_REF_SPLICE,CCDC14^ENSG00000175455.13,chr3:123961144:-,MCCC1^ENSG00000078070.10,chr3:183022554:-,YES_LDAS,0.0092,GT,1.7056,AG,1.6402,[INTRACHROMOSOMAL[chr3:59.05Mb]],INFRAME,tumor,True,False,IntraCHR
2,TCGA,OV,"Serous cystadenocarcinoma, NOS",TP,OV-24-1428-TP,TOLLIP::CHID1,2.0,0.0,2,ONLY_REF_SPLICE,TOLLIP^ENSG00000078902.14,chr11:1309466:-,CHID1^ENSG00000177830.16,chr11:884169:-,YES_LDAS,0.0092,GT,1.7056,AG,1.7819,[INTRACHROMOSOMAL[chr11:0.36Mb]],FRAMESHIFT,tumor,True,False,IntraCHR
3,TCGA,OV,"Serous cystadenocarcinoma, NOS",TP,OV-24-1428-TP,RCC1::PIK3R2,2.0,0.0,2,ONLY_REF_SPLICE,RCC1^ENSG00000180198.14,chr1:28508905:+,PIK3R2^ENSG00000268173.1,chr19:18168475:+,YES_LDAS,0.0092,GT,1.4295,AG,1.9329,[INTERCHROMOSOMAL[chr1--chr19]],INCL_NONCODING,tumor,True,False,INTERCHR
4,TCGA,OV,"Serous cystadenocarcinoma, NOS",TP,OV-24-1428-TP,CTNNB1::RAB5A,2.0,0.0,2,ONLY_REF_SPLICE,CTNNB1^ENSG00000168036.15,chr3:41239354:+,RAB5A^ENSG00000144566.9,chr3:19970548:+,YES_LDAS,0.0092,GT,1.7819,AG,1.9656,[INTRACHROMOSOMAL[chr3:21.21Mb]],INCL_NONCODING,tumor,True,False,IntraCHR


In [23]:
samples_with_insertions = insertions_data['sample_name'].drop_duplicates().tolist()

In [24]:
len(samples_with_insertions)

373

In [25]:
fusions_data = fusions_data.loc[fusions_data['sample_name'].isin(samples_with_insertions)]
fusions_data = fusions_data[ fusions_data.is_primary == True ]
fusions_data.shape

(8594, 26)

In [26]:
fusions_data

,data_class,tissue_type,tissue_subtype,tumor_subtype,sample_name,fusion_name,Jcount,Scount,Tcount,SpliceType,LeftGene,LeftBreakpoint,RightGene,RightBreakpoint,LargeAnchorSupport,FFPM,LeftBreakDinuc,LeftBreakEntropy,RightBreakDinuc,RightBreakEntropy,Annots,PROT_FUSION_TYPE,tumor_or_normal,is_primary,cosmic,structure_type
7192,TCGA,LIHC,"Hepatocellular carcinoma, NOS",TP,LIHC-DD-AACK-TP,ALDOA::ALDOB,1.0,1.0,2,ONLY_REF_SPLICE,ALDOA^ENSG00000149925.15,chr16:30069389:+,ALDOB^ENSG00000136872.16,chr9:101425627:-,NO_LDAS,0.0138,GT,1.9086,AG,1.5850,"[GTEx_recurrent_StarF2019,INTERCHROMOSOMAL[chr16--chr9]]",INFRAME,tumor,True,False,INTERCHR
7193,TCGA,LIHC,"Hepatocellular carcinoma, NOS",TP,LIHC-DD-AACK-TP,DNAJB11::SERPINA1,1.0,1.0,2,ONLY_REF_SPLICE,DNAJB11^ENSG00000090520.9,chr3:186585381:+,SERPINA1^ENSG00000197249.11,chr14:94379611:-,NO_LDAS,0.0138,GT,1.7968,AG,1.9656,[INTERCHROMOSOMAL[chr3--chr14]],FRAMESHIFT,tumor,True,False,INTERCHR
7194,TCGA,LIHC,"Hepatocellular carcinoma, NOS",TP,LIHC-DD-AACK-TP,G6PC::PPIA,1.0,1.0,2,ONLY_REF_SPLICE,G6PC^ENSG00000131482.8,chr17:42901106:+,PPIA^ENSG00000196262.12,chr7:44801287:+,NO_LDAS,0.0138,GT,1.8892,AG,1.8892,[INTERCHROMOSOMAL[chr17--chr7]],INFRAME,tumor,True,False,INTERCHR
7195,TCGA,LIHC,"Hepatocellular carcinoma, NOS",TP,LIHC-DD-AACK-TP,A2M::C4A,1.0,1.0,2,ONLY_REF_SPLICE,A2M^ENSG00000175899.13,chr12:9079244:-,C4A^ENSG00000244731.6,chr6:31995955:+,NO_LDAS,0.0138,GT,1.8062,AG,1.9086,[INTERCHROMOSOMAL[chr12--chr6]],INFRAME,tumor,True,False,INTERCHR
7196,TCGA,LIHC,"Hepatocellular carcinoma, NOS",TP,LIHC-DD-AACK-TP,CMTM8::CMTM7,1.0,1.0,2,ONLY_REF_SPLICE,CMTM8^ENSG00000170293.7,chr3:32239119:+,CMTM7^ENSG00000153551.12,chr3:32441840:+,NO_LDAS,0.0138,GT,1.9656,AG,1.9086,"[GTEx_recurrent_StarF2019,INTRACHROMOSOMAL[chr3:0.02Mb],NEIGHBORS[21346]]",INFRAME,tumor,True,False,READTHRU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399851,TCGA,CESC,"Squamous cell carcinoma, NOS",TP,CESC-WL-A834-TP,FGFR3::TACC3,1048.0,2289.0,3337,ONLY_REF_SPLICE,FGFR3^ENSG00000068078.16,chr4:1806934:+,TACC3^ENSG00000013810.17,chr4:1735731:+,YES_LDAS,63.2804,GT,1.8892,AG,1.9656,"[Cosmic,ChimerPub,YOSHIHARA_TCGA,TumorFusionsNAR2018,ChimerKB,ChimerSeq,CCLE_StarF2019,TCGA_StarF2019,GUO2018CR_TCGA,Klijn_CellLines,INTRACHROMOSOMAL[chr4:0.05Mb],LOCAL_REARRANGEMENT:+:[48131]]",INFRAME,tumor,True,True,LOCAL_REARRANGEMENT
399862,TCGA,CESC,"Squamous cell carcinoma, large cell, nonkeratinizing, NOS",TP,CESC-ZJ-AAXT-TP,FGFR3::TACC3,1516.0,3267.0,4783,ONLY_REF_SPLICE,FGFR3^ENSG00000068078.16,chr4:1806934:+,TACC3^ENSG00000013810.17,chr4:1739702:+,YES_LDAS,74.9950,GT,1.8892,AG,1.7819,"[Cosmic,ChimerPub,YOSHIHARA_TCGA,TumorFusionsNAR2018,ChimerKB,ChimerSeq,CCLE_StarF2019,TCGA_StarF2019,GUO2018CR_TCGA,Klijn_CellLines,INTRACHROMOSOMAL[chr4:0.05Mb],LOCAL_REARRANGEMENT:+:[48131]]",INFRAME,tumor,True,True,LOCAL_REARRANGEMENT
399888,TCGA,CESC,"Squamous cell carcinoma, NOS",TP,CESC-C5-A3HD-TP,PHTF1::PTPN22,1071.0,4974.0,6045,ONLY_REF_SPLICE,PHTF1^ENSG00000116793.14,chr1:113757699:-,PTPN22^ENSG00000134242.14,chr1:113819654:-,YES_LDAS,79.3254,GT,1.9219,AG,1.7232,"[TumorFusionsNAR2018,TCGA_StarF2019,INTRACHROMOSOMAL[chr1:0.05Mb],LOCAL_REARRANGEMENT:-:[54322]]",INFRAME,tumor,True,False,LOCAL_REARRANGEMENT
399942,TCGA,LIHC,"Hepatocellular carcinoma, NOS",TP,LIHC-BC-A10Y-TP,EIF4A2::KNG1,2242.0,12896.0,15138,ONLY_REF_SPLICE,EIF4A2^ENSG00000156976.13,chr3:186786273:+,KNG1^ENSG00000113889.10,chr3:186743705:+,YES_LDAS,198.3289,GT,1.8295,AG,1.9656,"[TumorFusionsNAR2018,TCGA_StarF2019,INTRACHROMOSOMAL[chr3:0.04Mb],LOCAL_REARRANGEMENT:+:[39251]]",INFRAME,tumor,True,False,LOCAL_REARRANGEMENT


# Examine each sample separately for nearby fusions

In [27]:

def extract_fusion_chromosome_coord(row, token):
    
    chromosome, coordinate = row[token].split(':')[0:2]

    row['Chromosome'] = chromosome
    row['Start'] = int(coordinate)
    row['End'] = int(coordinate) + 1
    
    return row
    
    
def extract_insertion_chromosome_coord(row):
    
    chromosome = row['humanchr']
    
    row['Chromosome'] = chromosome
    row['Start'] = row['coordA'] if row['chrA'] == chromosome else row['coordB']
    row['End'] = row['Start'] + 1
    
    return row


df_insertions_n_fusions = None

for sample in samples_with_insertions:
    print("-processing {}".format(sample))
    
    sample_insertions = insertions_data.loc[insertions_data['sample_name'] == sample]
    
    sample_fusions = fusions_data.loc[fusions_data['sample_name'] == sample]
    
    if len(sample_fusions) == 0:
        continue
    
    sample_fusions_left = sample_fusions.apply(extract_fusion_chromosome_coord,  axis=1, args=('LeftBreakpoint',))
    sample_fusions_right = sample_fusions.apply(extract_fusion_chromosome_coord, axis=1, args=('RightBreakpoint',) )
    
    sample_insertions = sample_insertions.apply(extract_insertion_chromosome_coord, axis=1)
    
    pr_insertions = pr.PyRanges(sample_insertions)
    
    pr_fusions_left = pr.PyRanges(sample_fusions_left)
    pr_fusions_right = pr.PyRanges(sample_fusions_right)
    
    pr_insertions_join_fusions_left = pr_insertions.nearest(pr_fusions_left)
    pr_insertions_join_fusions_right = pr_insertions.nearest(pr_fusions_right)
    
    df_insertions_n_fusions = pd.concat([df_insertions_n_fusions, pr_insertions_join_fusions_left.df, pr_insertions_join_fusions_right.df])
    
    #break
    
    
    
    

-processing CESC-C5-A1M6-TP
-processing CESC-C5-A7UC-TP
-processing CESC-LP-A5U2-TP
-processing CESC-DG-A2KM-TP
-processing LIHC-CC-A9FW-TP
-processing CESC-C5-A7X8-TP
-processing HNSC-P3-A6SW-TP
-processing CESC-VS-A953-TP
-processing HNSC-P3-A5Q5-TP
-processing LGG-QH-A6CS-TP
-processing CESC-EA-A44S-TP
-processing CESC-HG-A2PA-TP
-processing LIHC-BC-A10Y-TP
-processing CESC-EX-A8YF-TP
-processing LIHC-DD-AADK-TP
-processing CESC-C5-A7X3-TP
-processing HNSC-BA-4077-TP
-processing CESC-VS-A8QA-TP
-processing LIHC-5C-AAPD-TP
-processing CESC-EK-A2RC-TP
-processing HNSC-QK-A6IF-TP
-processing CESC-JW-A5VG-TP
-processing HNSC-CN-A6V7-TP
-processing LIHC-RC-A7SB-TP
-processing LIHC-DD-AACA-TP
-processing CESC-VS-A9U7-TP
-processing CESC-VS-A9V5-TP
-processing LIHC-DD-AACT-TP
-processing PAAD-H8-A6C1-TP
-processing CESC-EK-A3GN-TP
-processing LIHC-DD-AAD1-TP
-processing CESC-C5-A1BJ-TP
-processing LIHC-CC-A9FS-TP
-processing KIRC-DV-5573-TP
-processing LIHC-DD-AACS-TP
-processing HNSC-CV-5

In [28]:
df_insertions_n_fusions_selected = df_insertions_n_fusions[ df_insertions_n_fusions['Distance'] < 1e6 ]

df_insertions_n_fusions_selected 


,sample_name,contig,chrA,coordA,orientA,chrB,coordB,orientB,prelim.primary_brkpt_type,prelim.num_primary_reads,prelim.num_supp_reads,prelim.mean_hits,prelim.mean_min_per_id,prelim.mean_max_end_clipping,prelim.mean_min_anchor_len,prelim.total,prelim.adj_total,prelim.frac_reads_removed,entropyA,entropyB,flankA,flankB,splice_type,virus_brkend_grp,is_primary,split,span,total,flankA_fU,flankB_fU,TCGA,virus,humanchr,i_site,Chromosome,Start,End,data_class,tissue_type,tissue_subtype,tumor_subtype,sample_name_b,fusion_name,Jcount,Scount,Tcount,SpliceType,LeftGene,LeftBreakpoint,RightGene,RightBreakpoint,LargeAnchorSupport,FFPM,LeftBreakDinuc,LeftBreakEntropy,RightBreakDinuc,RightBreakEntropy,Annots,PROT_FUSION_TYPE,tumor_or_normal,is_primary_b,cosmic,structure_type,Start_b,End_b,Distance
0,CESC-LP-A5U2-TP,chr8~127742375~+~HPV16~6411~+,chr8,127742375,+,HPV16,6411,+,Span,1,48,1.000,99.4,1.429,46.490,49,49,0.0,1.800,1.859,ctcctaacctgtccaagccacctctcagacGA,AGacatttatttaatagggctggtactgttgg,.,HPV16:6411,True,0.0,47.0,47,1.000,0.963,CESC,HPV16,chr8,HPV16:chr8,chr8,127742375,127742376,TCGA,CESC,"Adenocarcinoma, endocervical type",TP,CESC-LP-A5U2-TP,PVT1::MYC,386.0,2529.0,2915,ONLY_REF_SPLICE,PVT1^ENSG00000249859.6,chr8:127794734:+,MYC^ENSG00000136997.13,chr8:127738248:+,YES_LDAS,45.2462,GT,1.8295,AG,1.6402,"[ChimerPub,ChimerSeq,INTRACHROMOSOMAL[chr8:0.05Mb],LOCAL_REARRANGEMENT:+:[53099]]",INCL_NONCODING,tumor,True,False,LOCAL_REARRANGEMENT,127794734,127794735,52359
0,CESC-LP-A5U2-TP,chr8~127742375~+~HPV16~6411~+,chr8,127742375,+,HPV16,6411,+,Span,1,48,1.000,99.4,1.429,46.490,49,49,0.0,1.800,1.859,ctcctaacctgtccaagccacctctcagacGA,AGacatttatttaatagggctggtactgttgg,.,HPV16:6411,True,0.0,47.0,47,1.000,0.963,CESC,HPV16,chr8,HPV16:chr8,chr8,127742375,127742376,TCGA,CESC,"Adenocarcinoma, endocervical type",TP,CESC-LP-A5U2-TP,PVT1::MYC,386.0,2529.0,2915,ONLY_REF_SPLICE,PVT1^ENSG00000249859.6,chr8:127794734:+,MYC^ENSG00000136997.13,chr8:127738248:+,YES_LDAS,45.2462,GT,1.8295,AG,1.6402,"[ChimerPub,ChimerSeq,INTRACHROMOSOMAL[chr8:0.05Mb],LOCAL_REARRANGEMENT:+:[53099]]",INCL_NONCODING,tumor,True,False,LOCAL_REARRANGEMENT,127738248,127738249,4127
0,CESC-VS-A953-TP,HPV16~3357~-~chr20~33658476~+,HPV16,3357,-,chr20,33658476,+,Split,13,91,1.192,99.6,4.394,43.192,104,104,0.0,1.933,1.922,ataatttcaggagaggatacttcgttgctgCT,ACcttctccaggaggcctcatcctgccagaac,GT-AG,HPV16:3357,Maybe,5.0,54.0,59,1.000,1.000,CESC,HPV16,chr20,HPV16:chr20,chr20,33658476,33658477,TCGA,CESC,"Squamous cell carcinoma, NOS",TP,CESC-VS-A953-TP,E2F1::PXMP4,1.0,1.0,2,ONLY_REF_SPLICE,E2F1^ENSG00000101412.12,chr20:33686004:-,PXMP4^ENSG00000101417.10,chr20:33707969:-,NO_LDAS,0.0248,GT,1.2310,AG,1.9329,"[INTRACHROMOSOMAL[chr20:0.02Mb],LOCAL_REARRANGEMENT:-:[16350]]",INFRAME,tumor,True,False,LOCAL_REARRANGEMENT,33686004,33686005,27528
1,CESC-VS-A953-TP,HPV16~637~-~chr20~33667157~+,HPV16,637,-,chr20,33667157,+,Span,7,171,1.000,99.9,4.511,43.444,178,178,0.0,1.832,1.820,cctcctcctctgagctgtcatttaattgctCA,CAgggctcgggcttcaccaaggcgggcttcgc,.,HPV16:637,True,0.0,112.0,112,0.889,0.815,CESC,HPV16,chr20,HPV16:chr20,chr20,33667157,33667158,TCGA,CESC,"Squamous cell carcinoma, NOS",TP,CESC-VS-A953-TP,E2F1::PXMP4,1.0,1.0,2,ONLY_REF_SPLICE,E2F1^ENSG00000101412.12,chr20:33686004:-,PXMP4^ENSG00000101417.10,chr20:33707969:-,NO_LDAS,0.0248,GT,1.2310,AG,1.9329,"[INTRACHROMOSOMAL[chr20:0.02Mb],LOCAL_REARRANGEMENT:-:[16350]]",INFRAME,tumor,True,False,LOCAL_REARRANGEMENT,33686004,33686005,18847
2,CESC-VS-A953-TP,HPV16~5777~-~chr20~33669145~+,HPV16,5777,-,chr20,33669145,+,Split,26,244,1.452,99.8,6.137,41.230,270,270,0.0,1.566,1.963,tgttaggttttttaataggaaaatagggatGT,TCgttctcactggtcacagtgatgtatgaagg,.,HPV16:5777,True,21.0,182.0,203,0.889,1.000,CESC,HPV16,chr20,HPV16:chr20,chr20,33669145,33669146,TCGA,CESC,"Squamous cell carcinoma, NOS",TP,CESC-VS-A953-TP,E2F1::PXMP4,1.0,1.0,2,ONLY_REF_SPLICE,E2F1^ENSG00000101412.12,chr20:33686004:-,PXMP4^ENSG00000101417.10,chr20:33707969:-,NO_LDAS,0.0248,GT,1.2310,AG,1.9329,"[INTRACHROMOSOMAL[chr20:0.

In [29]:
df_insertions_n_fusions_selected.to_csv("insertions_mapped_to_fusions.tsv", sep="\t", index=False, quoting=csv.QUOTE_NONE)

# restrict to hotspot insertions

In [30]:
insertions_at_hotspots = pd.read_csv("../InsertionHotspots/TCGA.hotspots.w_5neighbors.tsv", sep="\t")

In [31]:
insertions_at_hotspots

,sample,contig,human_chrom,human_coord,virus_genome,virus_coord,total,hotspot,hotspot_sample_counts,left_genes,right_genes
0,LIHC-DD-AAVP-TP,NC_003977_3182nt_Hepatitis_B_virus~1830~+~chr10~28655826~+,chr10,28655826,NC_003977_3182nt_Hepatitis_B_virus,1830,53,chr10:28655826,2,RN7SKP39;RP11-351M16.3;WAC-AS1;WAC;RNU4ATAC6P,TPRKBP1;RNU6-1067P;BAMBI;RP11-478H13.1;RNU6-270P
1,LIHC-BW-A5NP-TP,chr10~27906847~+~NC_003977_3182nt_Hepatitis_B_virus~1804~-,chr10,27906847,NC_003977_3182nt_Hepatitis_B_virus,1804,10,chr10:28655826,2,RAB18;MKX;MKX-AS1;RN7SKP132;RPL36AP55;<-(ARMC4)->,RP11-218D6.3;RP11-218D6.4;MPP7;MIR8086;AC022021.1
2,CESC-BI-A0VS-TP,HPV16~2915~+~chr10~34389298~+,chr10,34389298,HPV16,2915,85,chr10:35010252,4,RP13-259N13.2;LINC00838;RPL23P11;RP11-490O24.2;Y_RNA;<-(PARD3)->,TCEB2P4;RP11-8L18.3;RP11-8L18.2;PARD3-AS1;SS18L2P1
3,CESC-VS-A9UM-TP,chr10~35071340~+~HPV16~881~-,chr10,35071340,HPV16,881,269,chr10:35010252,4,PARD3-AS1;SS18L2P1;RNU6-847P;RNU6-193P;PRDX2P2;<-(CUL2)->,MIR3611;RP11-297A16.2;RNU7-77P;ATP6V1G1P4;CREM
4,CESC-EK-A2PI-TP,chr10~35016395~+~HPV16~881~-,chr10,35016395,HPV16,881,1089,chr10:35010252,4,PARD3-AS1;SS18L2P1;RNU6-847P;RNU6-193P;PRDX2P2;<-(CUL2)->,MIR3611;RP11-297A16.2;RNU7-77P;ATP6V1G1P4;CREM
...,...,...,...,...,...,...,...,...,...,...,...
398,HNSC-CN-A499-TP,chrY~11837814~+~HPV16~809~-,chrY,11837814,HPV16,809,16,chrY:11837814,2,RP11-295P22.3;MTND1P12;RP11-295P22.4;RP11-295P22.2;RCC2P1,ASS1P6;MXRA5Y;RPS24P1;ARSFP1;RN7SL702P
399,LIHC-DD-AADU-TP,chrY~11385744~+~NC_003977_3182nt_Hepatitis_B_virus~1774~-,chrY,11385744,NC_003977_3182nt_Hepatitis_B_virus,1774,33,chrY:11837814,2,DUX4L18;DUX4L19;AC134882.3;PABPC1P5;SLC9B1P1,ACTR3BP1;RP11-295P22.3;MTND1P12;RP11-295P22.4;RP11-295P22.2
400,LIHC-CC-A9FS-TP,chrY~57216743~+~NC_003977_3182nt_Hepatitis_B_virus~1823~-,chrY,57216743,NC_003977_3182nt_Hepatitis_B_virus,1823,146,chrY:57217140,3,IL9R;WASIR1;AJ271736.1;WASH6P;DDX11L16,NaN
401,CESC-EA-A3QD-TP,chrY~57197028~+~HPV16~2037~-,chrY,57197028,HPV16,2037,1942,chrY:57217140,3,AMDP1;DPH3P2;VAMP7;TCEB1P24;TRPC6P;<-(IL9R)->,WASIR1;AJ271736.1;WASH6P;DDX11L16


In [32]:
hotspot_insertion_contigs = insertions_at_hotspots['contig'].drop_duplicates()

In [33]:
hotspot_df_insertions_n_fusions_selected = df_insertions_n_fusions_selected[ df_insertions_n_fusions_selected['contig'].isin(hotspot_insertion_contigs)]

In [34]:
insertions_at_hotspots_w_fusions = insertions_at_hotspots.merge(hotspot_df_insertions_n_fusions_selected, 
                                                               left_on=['contig', 'sample'],
                                                                right_on=['contig', 'sample_name'], 
                                                                how='inner')

In [35]:
insertions_at_hotspots_w_fusions

,sample,contig,human_chrom,human_coord,virus_genome,virus_coord,total_x,hotspot,hotspot_sample_counts,left_genes,right_genes,sample_name,chrA,coordA,orientA,chrB,coordB,orientB,prelim.primary_brkpt_type,prelim.num_primary_reads,prelim.num_supp_reads,prelim.mean_hits,prelim.mean_min_per_id,prelim.mean_max_end_clipping,prelim.mean_min_anchor_len,prelim.total,prelim.adj_total,prelim.frac_reads_removed,entropyA,entropyB,flankA,flankB,splice_type,virus_brkend_grp,is_primary,split,span,total_y,flankA_fU,flankB_fU,TCGA,virus,humanchr,i_site,Chromosome,Start,End,data_class,tissue_type,tissue_subtype,tumor_subtype,sample_name_b,fusion_name,Jcount,Scount,Tcount,SpliceType,LeftGene,LeftBreakpoint,RightGene,RightBreakpoint,LargeAnchorSupport,FFPM,LeftBreakDinuc,LeftBreakEntropy,RightBreakDinuc,RightBreakEntropy,Annots,PROT_FUSION_TYPE,tumor_or_normal,is_primary_b,cosmic,structure_type,Start_b,End_b,Distance
0,CESC-C5-A1M8-TP,chr10~95756720~+~HPV16~881~-,chr10,95756720,HPV16,881,566,chr10:96153687,2,PDLIM1;SORBS1;RP11-7D5.2;ALDH18A1;TCTN3;<-(ENTPD1)->;<-(ENTPD1-AS1)->,RP11-429G19.3;C10orf131;RP11-248J23.6;RP11-248J23.5;CC2D2B,CESC-C5-A1M8-TP,chr10,95756720,+,HPV16,881,-,Split,56,594,1.065,98.5,3.718,43.749,650,650,0.0,1.941,1.963,cctgttgtgaaattagcaaaggcttctttgCT,ACctgcaggatcagccatggtagattatggtt,GT-AG,HPV16:881,True,45.0,521.0,566,1.000,0.963,CESC,HPV16,chr10,HPV16:chr10,chr10,95756720,95756721,TCGA,CESC,"Squamous cell carcinoma, NOS",TP,CESC-C5-A1M8-TP,SORBS1::ENTPD1-AS1,6.0,2.0,8,INCL_NON_REF_SPLICE,SORBS1^ENSG00000095637.19,chr10:95561320:-,ENTPD1-AS1^ENSG00000226688.5,chr10:95756719:-,NO_LDAS,0.0819,GT,1.5656,AG,1.9656,[INTRACHROMOSOMAL[chr10:0.19Mb]],INCL_NONCODING,tumor,True,False,IntraCHR,95561320,95561321,195400
1,CESC-C5-A1M8-TP,chr10~95756720~+~HPV16~881~-,chr10,95756720,HPV16,881,566,chr10:96153687,2,PDLIM1;SORBS1;RP11-7D5.2;ALDH18A1;TCTN3;<-(ENTPD1)->;<-(ENTPD1-AS1)->,RP11-429G19.3;C10orf131;RP11-248J23.6;RP11-248J23.5;CC2D2B,CESC-C5-A1M8-TP,chr10,95756720,+,HPV16,881,-,Split,56,594,1.065,98.5,3.718,43.749,650,650,0.0,1.941,1.963,cctgttgtgaaattagcaaaggcttctttgCT,ACctgcaggatcagccatggtagattatggtt,GT-AG,HPV16:881,True,45.0,521.0,566,1.000,0.963,CESC,HPV16,chr10,HPV16:chr10,chr10,95756720,95756721,TCGA,CESC,"Squamous cell carcinoma, NOS",TP,CESC-C5-A1M8-TP,SORBS1::ENTPD1-AS1,6.0,2.0,8,INCL_NON_REF_SPLICE,SORBS1^ENSG00000095637.19,chr10:95561320:-,ENTPD1-AS1^ENSG00000226688.5,chr10:95756719:-,NO_LDAS,0.0819,GT,1.5656,AG,1.9656,[INTRACHROMOSOMAL[chr10:0.19Mb]],INCL_NONCODING,tumor,True,False,IntraCHR,95756719,95756720,1
2,CESC-VS-A9UO-TP,chr11~102868070~+~HPV18~930~-,chr11,102868070,HPV18,930,1174,chr11:102949143,3,MMP10;WTAPP1;MMP1;AP000619.6;MMP3;<-(MMP12)->,BOLA3P1;RNU7-159P;MMP13;RP11-690D19.4;RP11-690D19.1,CESC-VS-A9UO-TP,chr11,102868070,+,HPV18,930,-,Split,279,1366,1.142,99.0,5.602,41.781,1645,1645,0.0,1.945,1.889,cgtgaacagcagtgaggaacaagtttgtgcCT,ACcttctggatcagccattgttgcttactgct,GT-AG,HPV18:930,True,171.0,1003.0,1174,0.963,1.000,CESC,HPV18,chr11,HPV18:chr11,chr11,102868070,102868071,TCGA,CESC,"Mucinous adenocarcinoma, endocervical type",TP,CESC-VS-A9UO-TP,MMP12::WTAPP1,167.0,722.0,889,ONLY_REF_SPLICE,MMP12^ENSG00000262406.2,chr11:102866315:-,WTAPP1^ENSG00000255282.5,chr11:102830732:+,YES_LDAS,9.2270,GT,1.3753,AG,1.5850,"[TCGA_StarF2019,INTRACHROMOSOMAL[chr11:0.03Mb],LOCAL_INVERSION:-:+:[25970]]",INCL_NONCODING,tumor,True,False,LOCAL_REARRANGEMENT,102866315,102866316,1755
3,CESC-VS-A9UO-TP,chr11~102868070~+~HPV18~930~-,chr11,102868070,HPV18,930,1174,chr11:102949143,3,MMP10;WTAPP1;MMP1;AP000619.6;MMP3;<-(MMP12)->,BOLA3P1;RNU7-159P;MMP13;RP11-690D19.4;RP11-690D19.1,CESC-VS-A9UO-TP,chr11,102868070,+,HPV18,930,-,Split,279,1366,1.142,99.0,5.602,41.781,1645,1645,0.0,1.945,1.889,cgtgaacagcagtgaggaacaagtttgtgcCT,ACcttctggatcagccattgttgcttactgct,GT-AG,HPV18:930,True,171.0,1003.0,1174,0.963,1.000,CESC,HPV18,chr11,HPV18:chr11,chr11,102868070,102868071,TCGA,CESC,"Mucinous adenocarcinoma, endocervical type",TP,CESC-VS-A9UO-TP,WTAPP1::MMP12,9.

In [36]:
insertions_at_hotspots_w_fusions.to_csv("insertions_mapped_to_fusions.at_hotspots.tsv", sep="\t", index=False, quoting=csv.QUOTE_NONE)

 m insertions_mapped_to_fusions.at_hotspots.tsv | print.pl 0 1 7 8 40 52 67  | sort -u  | sort -k3,3 > insertions_mapped_to_fusions.at_hotspots.abridged.tsv
 
then move the column headers back to the top manually.